In [ ]:
#如果在Windows或Linux環境上請跑此段。
#請先pip install gdown.
import gdown

url = 'https://drive.google.com/uc?export=download&id=10G2D9fm3UL92W2UuKdrw4bzJkDu0OflJ'
output = '/Code/NTUTDS/patent_documents/patentdoc.tar.gz'
#請使用自己的path
gdown.download(url, output, quiet=False)

In [21]:
#如果在Windows或Linux環境上請跑此段。
import os
import tarfile

def tar_extract(file_path):
    tar = tarfile.open(file_path, 'r:gz')
    tar.extractall()
    tar.close()

if __name__ == '__main__':
    file_path = '/Code/NTUTDS/patent_documents/patentdoc.tar.gz'
    tar_extract(file_path)
#請使用自己的path

In [ ]:
#老師原始程式碼，在Colab上請跑此段。
#!wget "https://drive.google.com/uc?export=download&id=10G2D9fm3UL92W2UuKdrw4bzJkDu0OflJ" -O patentdoc.tar.gz
#!tar xvfz patentdoc.tar.gz

In [ ]:
#資料抓取與處理
import pprint
from bs4 import BeautifulSoup
path = "patent_documents"
patent_csv = {}
patent_number = []
pub_date = [] 
country = []
app_type = []
app_date = []
patent_title = []
patent_abstract = []
applicant_name = []
inv_name = []
ipc_list = []
for filename in os.listdir(path):
  if filename.endswith(".txt"):
    with open(os.path.join(path, filename), 'r') as f:
      soup = BeautifulSoup(f.read(), 'xml')
      panum = soup.find("publication-reference").findChild("doc-number").contents # contents抓出來的東西資料類型是list...
      for pn in panum:
            patent_number.append(pn)
      pdate = soup.find("publication-reference").findChild("date").contents
      for pd in pdate:
             pub_date.append(pd)
      ctry = soup.find("publication-reference").findChild("country").contents
      for ct in ctry:
            country.append(ct)
      app_ref = soup.find("application-reference") #這個不要輸出
      atype = [app_ref['appl-type']]
      for at in atype:
            app_type.append(at)
      adate = app_ref.findChild("date").contents
      for ad in adate:
            app_date.append(ad)
      ptitle = [soup.find("invention-title").getText()] #要想辦法再把/n去掉
      for pt in ptitle:
            patent_title.append(pt)
      pabs = [soup.find("abstract").getText()] #要想辦法再把/n去掉
      for ab in pabs:
            patent_abstract.append(ab)
      #Agents = soup.find("orgname").getText() #如果文件沒有orgname這邊會return error
      us_applicants = soup.find("us-applicants") #這個不要輸出
      app_firstname = us_applicants.findChild("first-name").contents
      app_lastname = us_applicants.findChild("last-name").contents
      aname = [' '.join([a+b for a,b in zip(app_firstname,app_lastname)])] #記得之後要解決a跟b中間沒空格的問題 
      for an in aname:
               applicant_name.append(an)
      inventors = soup.find("inventors") #這個不要輸出
      inv_firstname = inventors.findChild("first-name").contents
      inv_lastname = inventors.findChild("last-name").contents
      iname = [' '.join([c+d for c,d in zip(inv_firstname,inv_lastname)])] #記得之後要解決a跟b中間沒空格的問題
      for ina in iname:
            inv_name.append(ina)
      ipcr_ref = soup.find("classification-ipcr") #這個不要輸出
      ipcr_section = ipcr_ref.findChild("section").contents
      ipcr_class = ipcr_ref.findChild("class").contents
      for class_num in ipcr_class:
            if len(str(class_num))==1:
                  class_num = str("0") + class_num
      ipcr_subclass = ipcr_ref.findChild("subclass").contents
      ipcs = [' '.join([d+e+f for d,e,f in zip(ipcr_section,ipcr_class,ipcr_subclass)])]
      for ip in ipcs:
            ipc_list.append(ip)

patent_csv = {"1.Patent Number":list(patent_number),"2.Publication Date":list(pub_date),"3.Application Date":list(app_date),"4.Country":list(country),"5.Application Type":list(app_type),"6.Title":list(patent_title),"7.Abstract":list(patent_abstract),"8.Applicant":list(applicant_name),"9.Inventor":list(inv_name),"IPCs":list(ipc_list)}

type(patent_csv)

patent_csv.pop('7.Abstract', None)

print(len(patent_csv))
pprint.pprint(patent_csv)


In [41]:
#使用Pandas匯出表格並存成CSV，也可以匯出給關聯式資料庫
import pandas as pd
df = pd.DataFrame.from_dict(patent_csv,orient='index').T
df.head(10)
df.to_csv("patent_table.csv",index=False)

In [ ]:
#Deal only claim texts
from bs4 import BeautifulSoup
import pprint
path = "patent_documents"
patent_docs_txt = {}
for filename in os.listdir(path):
  if filename.endswith(".txt"):
    with open(os.path.join(path, filename), 'r') as f:
      soup = BeautifulSoup(f.read(), 'xml')
      tag = soup.find_all("claims")
      patent_docs_txt[filename] = tag[0].text.lower().replace('\n',' ')

pprint.pprint(patent_docs_txt, depth=1, width=10000)

In [4]:
import nltk
import re

print(nltk.__version__)
nltk.download('punkt')
nltk.download('stopwords')
nltk_stopwords = nltk.corpus.stopwords.words('english')

def Patent_KW50(patent_text):
  tokens = nltk.word_tokenize(patent_text)
  fdist = nltk.FreqDist(tokens)
  common50 = fdist.most_common(50)
  common50_new1 = [(k,v) for k,v in common50 if not re.fullmatch(r"^.$", k)]
  common50_new2 = [(k,v) for k,v in common50_new1 if re.fullmatch(r"^[a-z]+$", k, flags=re.IGNORECASE)]
  stopword = list(nltk_stopwords)[:100]
  common50_new3 = dict([(k,v) for k,v in common50_new2 if not k in stopword])
  return common50_new3

patent_docs_KW = {}
for doc_name, txt in patent_docs_txt.items():
  patent_docs_KW[doc_name] = Patent_KW50(txt)

pprint.pprint(patent_docs_KW, width=1000)

3.6.5


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\謝安\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\謝安\AppData\Roaming\nltk_data...


{'US08438662-20130514.txt': {'adf': 10, 'assembly': 34, 'behind': 11, 'claim': 39, 'comprises': 10, 'comprising': 15, 'configured': 10, 'cover': 17, 'different': 13, 'dimensions': 13, 'face': 47, 'filter': 9, 'first': 57, 'front': 30, 'interchangeable': 14, 'least': 19, 'lens': 160, 'lenses': 18, 'matching': 10, 'one': 9, 'opening': 24, 'plate': 19, 'portion': 11, 'properties': 10, 'protector': 31, 'releasably': 11, 'retainer': 10, 'second': 53, 'secured': 20, 'securing': 13, 'shade': 10, 'substantially': 21, 'transparent': 10, 'wearer': 14, 'wherein': 32},
 'US08438663-20130514.txt': {'assembly': 27, 'back': 17, 'behind': 13, 'claim': 28, 'comprises': 11, 'comprising': 14, 'configured': 15, 'cover': 27, 'different': 10, 'extending': 8, 'face': 39, 'first': 27, 'frame': 14, 'front': 14, 'interchangeable': 9, 'least': 17, 'lens': 94, 'lenses': 15, 'magnifying': 10, 'one': 9, 'opening': 14, 'outer': 15, 'plate': 10, 'portion': 9, 'protector': 26, 'releasable': 7, 'releasably': 9, 'second

[nltk_data]   Unzipping corpora\stopwords.zip.


In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from itertools import islice

plt.clf()

for doc_name, KW_stat in patent_docs_KW.items():
  common20 = dict(islice(KW_stat.items(), 20))
  print(common20)
  x = np.arange(20)
  y = common20.values()
  plt.title( doc_name, fontsize=8, color='g')
  plt.xticks(x, common20.keys(),  rotation='vertical')
  plt.bar(x, y, color='y')
  plt.show()